In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib as mpl
from matplotlib import rc
rc('text',usetex=True)
rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
import matplotlib.pyplot as plt
% matplotlib inline
% config InlineBackend.figure_format = 'svg'
% config InlineBackend.rc = {'figure.figsize': (5,3.5)}

In [3]:
import numpy as np
from numpy import sqrt,log,array as ar
import numpy.linalg as la
import cvxpy as cvx
from helper import *
from cd.model.utility import LipschitzExpUtility as leu, RiskNeutralUtility as rnu
from cd.model.distrs import RademacherDistribution,UniformDistribution
import cd.model.synth_data as synth
import scaler
from itertools import product

# Ajout d'Information

In [4]:
p = 50
R = RademacherDistribution()
Xs = [RademacherDistribution() for _ in range(p)]

In [5]:
M = synth.GaussianMarket(Xs,R)
t_true = M.sample_t(1_000)

In [40]:
m = 150
n = 5
u = leu(1)
ts = create_sample(t_true,n,m)

In [41]:
ps = np.arange(2,51)
erroru = np.zeros(len(ps))
boundu = np.zeros(len(ps))
errorce = np.zeros(len(ps))
boundce = np.zeros(len(ps))
bfor i,p in enumerate(ps):
    if i%5 == 0: print(i)
    t = ts[:,:,:p]
    qs = solve(t,u,lamb=1)
    inerror = in_error(qs,u,t,lamb=1)
    outerror = out_error(qs,u,t_true,lamb=1)
    erroru[i] = error_u(inerror,outerror)
    errorce[i] = error_ce(inerror,outerror,u)
    boundu[i] = bound_genu(n,sqrt(p),lamb=1)
    boundce[i] = bound_gence(n,sqrt(p),u,inerror,outerror,lamb=1)    

45


40


35


30


25


20


15


10


5


0


### Utilité

In [102]:
plt.plot(ps,erroru,label='Erreur maximale empirique')
plt.plot(ps[0],erroru[0],'k--',label="Borne de g\\'en\\'eralisation")
plt.xlabel('$p$')
plt.tick_params('y',colors='C0')
plt.axis(ymax=2*max(erroru))
yo = plt.axes().get_yticks()
plt.axes().set_yticks([el for el in yo if el <= max(erroru)]+[0.15])
plt.legend(loc=2)
plt2 = plt.twinx()
plt.plot(ps,boundu,'k--')
yo =plt2.get_yticks()
plt2.set_yticks([0]+[el for el in yo if el >= min(boundu)])
plt2.axis(ymin=-120);
plt.savefig('fig/bound_pugen.pdf',bbox_inches='tight',pad_inches=0)

### Équivalent certain

In [105]:
plt.plot(ps,errorce,label='Erreur maximale empirique')
plt.plot(ps[0],errorce[0],'k--',label="Borne de g\\'en\\'eralisation")
plt.xlabel('$p$')
plt.tick_params('y',colors='C0')
plt.axis(ymax=2*max(errorce))
yo = plt.axes().get_yticks()
plt.axes().set_yticks([el for el in yo if el <= max(errorce)]+[0.15])
plt.legend(loc=2)
plt2 = plt.twinx()
plt.plot(ps,boundce,'k--')
yo =plt2.get_yticks()
plt2.set_yticks([el for el in yo if el >= min(boundce)])
plt2.axis(ymin=-120);
plt.savefig('fig/bound_pugence.pdf',bbox_inches='tight',pad_inches=0)


## Sous optimalité

In [4]:
p = 50
R = RademacherDistribution()
Xs = [RademacherDistribution() for _ in range(p)]
u = leu(1)

In [5]:
M = synth.GaussianMarket(Xs,R,corr_vector=[0 for _ in range(p)])
t_true = M.sample_t(1_000)

In [6]:
q_true = solve(t_true,u=u,lamb=0)
eu_true = u(t_true@q_true[0]).mean()

In [7]:
m = 50
n = 3
ts = create_sample(t_true,n,m)

In [8]:
ps = np.arange(2,51)
soerror = np.zeros(len(ps))
n_qtrues = np.zeros(len(ps))
eu_trues = np.zeros(len(ps))
for i,p in enumerate(ps):
    if i%5 == 0: print(i)
    t = ts[:,:,:p]
    q_true = solve(t_true[:,:p],u=u,lamb=0)[0]
    n_qtrues[i] = la.norm(q_true)**2
    eu_trues[i] = u(t_true[:,:p]@q_true).mean()
    qs = solve(t,u,lamb=1)
    soerror[i] = out_error(qs,u,t_true,lamb=1).min()

0


5


10


15


20


25


30


35


40


45


### Utilité

In [21]:
tol = log(1/0.01)
om = 4*ps*(32+tol)/n
plt.plot(ps,soerror,label="Utilit\\'e hors \\'echantillon")
plt.plot(ps[0],soerror[0],'k--',label="Borne th\\'eorique")
plt.tick_params('y',colors='C0')
plt.axis(ymin=-max(soerror) + 2*min(soerror))
yo = plt.axes().get_yticks()
plt.axes().set_yticks([el for el in yo if el >= min(soerror)])
plt.xlabel('$p$')
plt.legend()
plt2 = plt.twinx()
bound = eu_trues - 4*om - sqrt(2*ps*om) - 0.5*n_qtrues
plt.plot(ps,bound,'k--')
plt.axis(ymax=-min(bound)  + 2*max(bound));
yo =plt2.get_yticks()
plt2.set_yticks([el for el in yo if el <= max(bound)]);
plt.savefig('fig/bound_puso.pdf',bbox_inches='tight',pad_inches=0)


### Équivalent certain

In [22]:
tol = log(1/0.05)
om = 4*ps*(32+tol)/n
plt.plot(ps,u.inverse(soerror),label="Utilit\\'e hors \\'echantillon")
plt.plot(ps[0],soerror[0],'k--',label="Borne th\\'eorique")
plt.tick_params('y',colors='C0')
plt.axis(ymin=-max(soerror) + 2*min(soerror))
yo = plt.axes().get_yticks()
plt.axes().set_yticks([el for el in yo if el >= min(soerror)])
plt.xlabel('$p$')
plt.legend()
plt2 = plt.twinx()
bound = u.inverse(eu_trues) - u.subinverse(soerror)*(4*om + sqrt(2*ps*om) + 0.5*n_qtrues)
plt.plot(ps,bound,'k--')
plt.axis(ymax=-min(bound)  + 2*max(bound));
yo =plt2.get_yticks()
plt2.set_yticks([el for el in yo if el <= max(bound)]);
plt.savefig('fig/bound_pceso.pdf',bbox_inches='tight',pad_inches=0)
